In [91]:
DL = '/Users/damiansp/Downloads'

In [92]:
!rm -rf $DL/*

In [93]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [100]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADBE', 'ADYEY', 'AMD', 'AMZN', 'ANET',
    'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'CRWD', 'DDOG', 'DOCS',
    'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'GH', 'GMED', 'GOOG',
    'IDXX', 'INTG', 'ISRG', 'JD', 'LSPD', 'LULU', 'MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NVCR', 'NVDA', 'OKTA', 'OPEN', 'PAC', 'PANW',
    'PAYC', 'PINS', 'PYPL', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'RIVN',
    'ROKU', 'RVLV', 'SE', 'SHOP', 'SNOW', 'TASK', 'TDOC', 'TEAM',
    'TREX', 'TTD', 'TTWO', 'TWLO', 'TWTR', 'TYL', 'UPST', 'VEEV', 'VLD',
    'XPEV', 'ZEN', 'ZM']
LINGERERS = [
    'AMEH', 'APT', 'BYRN', 'CIZN', 'CUBE', 'EP', 'FCUV', 'GCBC', 'GME',
    'INTZ', 'KREF', 'MCVT', 'MICS', 'PETV', 'POWW', 'PRPH', 'RA', 'RVP',
    'WAVD', 'WKHS', 'VIRT', 'ZYXI']
BEST_WEIGHTED = [
    'ABM', 'AMRK', 'CELH', 'CTBB', 'CTDD', 'EVA', 'EXR', 'FCPT', 'FND',
    'GBOX', 'INSW', 'KNSL', 'LFMD', 'MNMD', 'MYOV', 'NSA', 'OMQS',
    'RRR', 'SBOW', 'STOR']
SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))

In [101]:
!ls $DL

DiffusionBee-0.1.0-arm64.dmg
Portfolio_Positions_Sep-17-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_17(1).xls
Positions_damiansp_Stocks_2022_09_17.xls


In [102]:
#!head -5 $DL/Positions.csv

In [103]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [104]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [105]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABM
ABMD
ABNB
ADBE


### E*Trade

In [106]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [107]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
AAPL,1356.3
ABMD,1293.3500000000001
ABNB,356.25
ADBE,599
ADYEY,4075.71


In [108]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
AAPL,1356
ABMD,1293
ABNB,356
ADBE,599
ADYEY,4076


In [109]:
#et_df.drop(index=['98986CONT'], inplace=True)
#et_df.head()

In [110]:
et_df.loc[et_df.ET == '--']
#et_df.loc['NVCR', 'ET'] = 79.24

,ET


In [111]:
check_symbols(et_df)

All clear


In [112]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,1356
ABM,0
ABMD,1293
ABNB,356
ADBE,599


### Fidelity

In [113]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Sep-16-2022.csv'

In [114]:
!ls $DL

DiffusionBee-0.1.0-arm64.dmg
Portfolio_Positions_Sep-17-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_17(1).xls
Positions_damiansp_Stocks_2022_09_17.xls


In [116]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-16-", "-17-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,720.47,$1.00,$0.00,$720.47,NaN,NaN,NaN,NaN,1.98%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,4.00,$258.67,-$5.05,$1034.68,-$20.20,-1.92%,+$15.99,+1.56%,2.84%,$1018.69,$254.67,Cash
2,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,2.00,$118.75,-$6.29,$237.50,-$12.58,-5.04%,-$48.00,-16.82%,0.65%,$285.50,$142.75,Cash
3,218520300,ROLLOVER IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,13.00,$13.19,-$0.046,$171.47,-$0.60,-0.35%,-$43.22,-20.14%,0.47%,$214.69,$16.51,Cash
4,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,19.00,$76.51,-$0.15,$1453.69,-$2.85,-0.20%,-$418.21,-22.35%,4.00%,$1871.90,$98.52,Cash


In [117]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,ABMD,1035
2,ROLLOVER IRA,ABNB,238
3,ROLLOVER IRA,ADYEY,171
4,ROLLOVER IRA,AMD,1454
5,ROLLOVER IRA,AMZN,247


In [118]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
ABMD,1035
ABNB,238
ADYEY,171
AMD,1454
AMZN,247


In [119]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [120]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,1356,0,6028,0
ABM,0,0,0,0
ABMD,1293,1035,3363,0
ABNB,356,238,594,0
ADBE,599,0,0,0


### TD Ameritrade

In [121]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [128]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

In [129]:
# Stock (default) view
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1])
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [130]:
td = parse_file(path)
td.head()

,TD
AMEH,117.72
AMRK,152.40
APT,84.84
BYRN,322.00
CELH,100.70


In [131]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,1356,0,6028,0,0
ABM,0,0,0,0,0
ABMD,1293,1035,3363,0,0
ABNB,356,238,594,0,0
ADBE,599,0,0,0,0


In [132]:
out.to_csv(f'{DL}/clean.csv')

In [133]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [47]:
!open $DL/mw.csv

The file /Users/damiansp/Downloads/mw.csv does not exist.


In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*